In [1]:
import pandas as pd
import numpy as np
from lifelines.utils import concordance_index

## Getting counts of patients/slices in train/valid/test datasets

In [23]:
train_info = pd.read_csv('/Data/train_info_fold1.csv')
valid_info = pd.read_csv('/Data/valid_info_fold1.csv')
test_info = pd.read_csv('/Data/Labels/HDFS_test_tumors.csv')

In [24]:
patient_valid_info = valid_info.loc[valid_info['Slice_Num']==1]
# patient_valid_info.reset_index(inplace=True)
patient_valid_info.to_csv('/Data/patient_valid_info.csv', index=False)

In [25]:
patient_train_info = train_info.loc[train_info['Slice_Num']==1]
# patient_train_info.reset_index(inplace=True)
patient_train_info.to_csv('/Data/patient_train_info.csv', index=False)

In [26]:
patient_test_info = test_info.loc[test_info['Slice_Num']==1]
# patient_test_info.reset_index(inplace=True)
patient_test_info.to_csv('/Data/patient_test_info.csv', index=False)

In [27]:
total_train = pd.read_csv('/Data/Labels/HDFS_train_tumors.csv')
patient_total_train = total_train.loc[total_train['Slice_Num']==1]
patient_total_train.shape
patient_total_train.to_csv('/Data/correct_train_patients.csv', index=False)

## Calculating c-index for each cancer types predictions

In [ ]:
from hdfs_models import *

In [3]:
# test_preds_0134 = pd.read_csv('/Data/Output/HDFSModel2/Liver/2022_02_22_2020_train/test_predictions.csv')
# ICC_preds_0134 = test_preds_0134[test_preds_0134['Slice_File_Name'].str.contains("ICC")]
# HCC_preds_0134 = test_preds_0134[test_preds_0134['Slice_File_Name'].str.contains("hcc")]
# MCRC_preds_0134 = test_preds_0134[test_preds_0134['Slice_File_Name'].str.contains("preop")]

# ICC_preds_0134.to_csv('/Data/ICC_preds_0134.csv', index=False)
# HCC_preds_0134.to_csv('/Data/HCC_preds_0134.csv', index=False)
# MCRC_preds_0134.to_csv('/Data/MCRC_preds_0134.csv', index=False)

In [22]:
test_preds_0134 = pd.read_csv('/Data/Output/HDFSModel2/Tumor/2022_03_28_1903_train/test_predictions.csv')
ICC_preds_0134 = test_preds_0134[test_preds_0134['Slice_File_Name'].str.contains("ICC")]
HCC_preds_0134 = test_preds_0134[test_preds_0134['Slice_File_Name'].str.contains("hcc")]
MCRC_preds_0134 = test_preds_0134[test_preds_0134['Slice_File_Name'].str.contains("preop")]

np_all_preds = np.array(test_preds_0134['Prediction'])
np_ICC_preds = np.array(ICC_preds_0134['Prediction'])
np_HCC_preds = np.array(HCC_preds_0134['Prediction'])
np_MCRC_preds = np.array(MCRC_preds_0134['Prediction'])

all_ghci = gh_c_index(np_all_preds)
ICC_ghci = gh_c_index(np_ICC_preds)
HCC_ghci = gh_c_index(np_HCC_preds)
MCRC_ghci = gh_c_index(np_MCRC_preds)

print("CNN Prediction Scores")
print("Total GHCI:", all_ghci)
print("HCC GHCI: ", HCC_ghci)
print("MCRC GHCI: ", MCRC_ghci)
print("ICC GHCI: ", ICC_ghci)

# print("All n:", test_preds_0134.shape)
# print("HCC n:", np_HCC_preds.shape)
# print("MCRC n:", np_MCRC_preds.shape)
# print("ICC n:", np_ICC_preds.shape)

ICC_cind = concordance_index(np.array(ICC_preds_0134['Time']), np_ICC_preds, np.array(ICC_preds_0134['Event']))
HCC_cind = concordance_index(np.array(HCC_preds_0134['Time']), np_HCC_preds, np.array(HCC_preds_0134['Event']))
MCRC_cind = concordance_index(np.array(MCRC_preds_0134['Time']), np_MCRC_preds, np.array(MCRC_preds_0134['Event']))
total_cind = concordance_index(np.array(test_preds_0134['Time']), np_all_preds, np.array(test_preds_0134['Event']))

print("\nTotal c-index:", total_cind)
print("HCC C-index:", HCC_cind)
print("MCRC C-index:", MCRC_cind)
print("ICC C-index:", ICC_cind)

CNN Prediction Scores
Total GHCI: 0.6299562527886206
HCC GHCI:  0.6628711343869379
MCRC GHCI:  0.5966570741252614
ICC GHCI:  0.6368397784656598

Total c-index: 0.6012166208544684
HCC C-index: 0.5953355883923803
MCRC C-index: 0.6613858398400664
ICC C-index: 0.5838111298482294


In [23]:
print("CPH Predictions")
cph_preds = pd.read_excel("/Data/FeatureSelection/HCC_MCRC_ICC_HDFS_90_10/CPH_test_liver_predictions.xlsx")

HCC_cph_preds = cph_preds[cph_preds['Cancer_Type']==0]
MCRC_cph_preds = cph_preds[cph_preds['Cancer_Type']==1]
ICC_cph_preds = cph_preds[cph_preds['Cancer_Type']==2]

HCC_ghci = gh_c_index(np.array(HCC_cph_preds['Prediction']))
MCRC_ghci = gh_c_index(np.array(MCRC_cph_preds['Prediction']))
ICC_ghci = gh_c_index(np.array(ICC_cph_preds['Prediction']))

# HAVE TO SWITCH THE SIGN ON THE PREDICTIONS FOR CPH MODEL
HCC_cind = concordance_index(np.array(HCC_cph_preds['Prediction']), -np.array(HCC_cph_preds['HDFS_Time']), np.array(HCC_cph_preds['HDFS_Code']))
MCRC_cind = concordance_index(np.array(MCRC_cph_preds['Prediction']), -np.array(MCRC_cph_preds['HDFS_Time']), np.array(MCRC_cph_preds['HDFS_Code']))
ICC_cind = concordance_index(np.array(ICC_cph_preds['Prediction']), -np.array(ICC_cph_preds['HDFS_Time']), np.array(ICC_cph_preds['HDFS_Code']))


print("HCC GHCI: ", HCC_ghci)
print("MCRC GHCI: ", MCRC_ghci)
print("ICC GHCI: ", ICC_ghci)

print("\nHCC C-index:", HCC_cind)
print("MCRC C-index:", MCRC_cind)
print("ICC C-index:", ICC_cind)


all_cind = concordance_index(np.array(cph_preds['HDFS_Time']), -np.array(cph_preds['Prediction']), np.array(cph_preds['HDFS_Code']))
print("\nAll c-index:", all_cind)

CPH Predictions
HCC GHCI:  0.6024374932735151
MCRC GHCI:  0.6039925092244356
ICC GHCI:  0.6020811945659768

HCC C-index: 0.4772727272727273
MCRC C-index: 0.13636363636363635
ICC C-index: 0.5876288659793815

All c-index: 0.5733882030178327


In [22]:
print("RSF Predictions")
rsf_preds = pd.read_excel("/Data/FeatureSelection/HCC_MCRC_ICC_HDFS_90_10/RSF_test_liver_predictions_90_10_040822.xlsx")

HCC_rsf_preds = rsf_preds[rsf_preds['Cancer_Type']==0]
MCRC_rsf_preds = rsf_preds[rsf_preds['Cancer_Type']==1]
ICC_rsf_preds = rsf_preds[rsf_preds['Cancer_Type']==2]

HCC_ghci = gh_c_index(np.array(HCC_rsf_preds['Prediction']))
MCRC_ghci = gh_c_index(np.array(MCRC_rsf_preds['Prediction']))
ICC_ghci = gh_c_index(np.array(ICC_rsf_preds['Prediction']))

HCC_cind = concordance_index(np.array(HCC_rsf_preds['HDFS_Time']), np.array(HCC_rsf_preds['Prediction']), np.array(HCC_rsf_preds['HDFS_Code']))
MCRC_cind = concordance_index(np.array(MCRC_rsf_preds['HDFS_Time']), np.array(MCRC_rsf_preds['Prediction']), np.array(MCRC_rsf_preds['HDFS_Code']))
ICC_cind = concordance_index(np.array(ICC_rsf_preds['HDFS_Time']), np.array(ICC_rsf_preds['Prediction']), np.array(ICC_rsf_preds['HDFS_Code']))


print("HCC GHCI: ", HCC_ghci)
print("MCRC GHCI: ", MCRC_ghci)
print("ICC GHCI: ", ICC_ghci)

print("\nHCC C-index:", HCC_cind)
print("MCRC C-index:", MCRC_cind)
print("ICC C-index:", ICC_cind)

all_cind = concordance_index(np.array(rsf_preds['HDFS_Time']), np.array(rsf_preds['Prediction']), np.array(rsf_preds['HDFS_Code']))
print("\nAll c-index:", all_cind)


RSF Predictions
HCC GHCI:  0.6545398420802158
MCRC GHCI:  0.8881659785141696
ICC GHCI:  0.9339280237099136

HCC C-index: 0.6481481481481481
MCRC C-index: 0.49390243902439024
ICC C-index: 0.62890625

All c-index: 0.5521262002743484


In [15]:
all_cind = concordance_index(np.array(cph_preds['HDFS_Time']), -np.array(cph_preds['Prediction']), np.array(cph_preds['HDFS_Code']))
print(all_cind)

0.6844993141289437


In [10]:
HCC_cph_preds

,ScoutID,HDFS_Time,HDFS_Code,Cancer_Type,Prediction
0,029-hcc,18.0,0,0,-0.331360
1,038-hcc,8.0,0,0,-0.092027
2,047-hcc,133.0,0,0,-0.309715
3,055-hcc,11.0,0,0,-0.501277
4,064-hcc,50.0,0,0,-0.163785
5,065-hcc,18.0,1,0,-0.321827
6,072-hcc,5.0,1,0,0.062120
7,107-hcc,15.0,1,0,-0.289166
8,114-hcc,120.0,0,0,-0.554152
9,116-hcc,98.0,0,0,-0.222633


# Old Code

In [8]:
ICC_cind = concordance_index(np_ICC_preds, np.array(ICC_preds_0134['Time']), np.array(ICC_preds_0134['Event']))
HCC_cind = concordance_index(np_HCC_preds, np.array(HCC_preds_0134['Time']), np.array(HCC_preds_0134['Event']))
MCRC_cind = concordance_index(np_MCRC_preds, np.array(MCRC_preds_0134['Time']), np.array(MCRC_preds_0134['Event']))
total_cind = concordance_index(np.array(test_preds_0134['Prediction']), np.array(test_preds_0134['Time']), np.array(test_preds_0134['Event']))

print("Total c-index:", total_cind)
print("HCC C-index:", HCC_cind)
print("MCRC C-index:", MCRC_cind)
print("ICC C-index:", ICC_cind)

HCC C-index: 0.7611696274170728
MCRC C-index: 0.744561624939823
ICC C-index: 0.6267529665587918
Total c-index: 0.7131675612170434


In [ ]:
print("All n:", test_preds_0134.shape)
print("HCC n:", np_HCC_preds.shape)
print("MCRC n:", np_MCRC_preds.shape)
print("ICC n:", np_ICC_preds.shape)

In [4]:
test_preds_0134.head(50)

,Slice_File_Name,Prediction,Time,Event
0,005_ICCrecurrence_Tumor_Slice_1.bin,-0.066470,3.700000,1.0
1,005_ICCrecurrence_Tumor_Slice_10.bin,-0.551257,3.700000,1.0
2,005_ICCrecurrence_Tumor_Slice_11.bin,-0.323575,3.700000,1.0
3,005_ICCrecurrence_Tumor_Slice_2.bin,-0.938154,3.700000,1.0
4,005_ICCrecurrence_Tumor_Slice_3.bin,-1.236490,3.700000,1.0
5,005_ICCrecurrence_Tumor_Slice_4.bin,-0.743327,3.700000,1.0
6,005_ICCrecurrence_Tumor_Slice_5.bin,-0.531766,3.700000,1.0
7,005_ICCrecurrence_Tumor_Slice_6.bin,-0.444448,3.700000,1.0
8,005_ICCrecurrence_Tumor_Slice_7.bin,-0.213274,3.700000,1.0
9,005_ICCrecurrence_Tumor_Slice_8.bin,-0.506335,3.700000,1.0
